In [1]:
import os
import pandas as pd
import numpy as np

# Loading Dataset

In [2]:
comment_file = './data/stock_comments_seg.csv'
data_path = './data'
pos_corpus = 'positive.txt'
neg_corpus = 'negative.txt'

# Concact positive and negative corpus together, and return with corresponding tokens
def load_dataset_tokenized():
    pos_file = os.path.join(data_path, pos_corpus)
    neg_file = os.path.join(data_path, neg_corpus)

    pos_sents = []
    with open(pos_file, 'r', encoding='utf-8') as f:
        for line in f:
            tokens = line.split(' ')
            sent = []
            for t in tokens:
                if t.strip():
                    sent.append(t.strip())
            pos_sents.append(sent)

    neg_sents = []
    with open(neg_file, 'r', encoding='utf-8') as f:
        for line in f:
            tokens = line.split(' ')
            sent = []
            for t in tokens:
                if t.strip():
                    sent.append(t.strip())
            neg_sents.append(sent)
            
    # Ensure same number of pos and neg reviews 
    balance_len = min(len(pos_sents), len(neg_sents))

    texts = pos_sents + neg_sents
    labels = [1] * balance_len + [0] * balance_len

    return texts, labels

# Evalauting Models

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold
from sklearn import svm
from sklearn import naive_bayes
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import metrics
from sklearn.utils import shuffle

In [4]:
def simple_return(f):
    return f

def KFold_validation(clf, X, y):
    
    # initialize output
    acc = []
    precision, recall, f1_score = [], [], []

    kf = KFold(n_splits = 5, shuffle = True, random_state = 30)
    for train, test in kf.split(X):
        X_train = [X[i] for i in train]
        X_test = [X[i] for i in test]
        y_train = [y[i] for i in train]
        y_test = [y[i] for i in test]

        vectorizer = TfidfVectorizer(analyzer = 'word',
                                     tokenizer = simple_return,
                                     preprocessor = simple_return,
                                     token_pattern =None)

        vectorizer.fit(X_train)
        # vectorize train and test sets
        X_train = vectorizer.transform(X_train)
        X_test = vectorizer.transform(X_test)
        # training the model
        clf.fit(X_train, y_train)
        # get prediction
        preds = clf.predict(X_test)

        acc.append(metrics.accuracy_score(y_test, preds))
        precision.append(metrics.precision_score(y_test, preds, pos_label = 1))
        recall.append(metrics.recall_score(y_test, preds, pos_label = 1))
        f1_score.append(metrics.f1_score(y_test, preds, pos_label = 1))

    return (np.mean(acc), np.mean(precision), np.mean(recall), np.mean(f1_score))

In [5]:
# Evaluate different models, return a DataFrame containing the performance metrics
def evaluate_models():

    X, y = load_dataset_tokenized()

    cols = ['metrics', 'accuracy',  'precision', 'recall', 'f1_score']
    scores = []

    classifiers = [
        ('LinearSVC', svm.LinearSVC()),
        ('LogisticReg', LogisticRegression()),
        ('SGD', SGDClassifier()),
        ('MultinomialNB', naive_bayes.MultinomialNB()),
        ('KNN', KNeighborsClassifier()),
        ('DecisionTree', DecisionTreeClassifier()),
        ('RandomForest', RandomForestClassifier()),
        ('AdaBoost', AdaBoostClassifier(base_estimator=LogisticRegression()))]

    for name, clf in classifiers:
        score = KFold_validation(clf, X, y)
        row = [name]
        row.extend(score)
        scores.append(row)

    df = pd.DataFrame(scores, columns = cols).T
    df.columns = df.iloc[0]
    df.drop(df.index[[0]], inplace = True)
    df = df.apply(pd.to_numeric, errors = 'ignore')

    return df

In [6]:
# Get model performances
evaluate_models()

metrics,LinearSVC,LogisticReg,SGD,MultinomialNB,KNN,DecisionTree,RandomForest,AdaBoost
accuracy,0.883330,0.882462,0.884524,0.877469,0.817344,0.802583,0.843716,0.826463
precision,0.884507,0.881430,0.887860,0.883472,0.809008,0.819548,0.869269,0.827792
recall,0.882010,0.883984,0.880231,0.869941,0.831042,0.776016,0.809178,0.842726
f1_score,0.883209,0.882670,0.884008,0.876548,0.819673,0.797150,0.838098,0.828838


# Apply Bagging Algorithm

In [7]:
# only keep frist 5 models with best performances
# save the prediction in another file
def predict_with_bagging():

    # Loading training data
    X, y = load_dataset_tokenized()
    vectorizer = TfidfVectorizer(analyzer = 'word',
                                 tokenizer = simple_return,
                                 preprocessor = simple_return,
                                 token_pattern = None)

    X = vectorizer.fit_transform(X)

    # Loading review file
    df = pd.read_csv(comment_file)
    df.dropna(inplace = True)
    df.reset_index(drop = True, inplace = True)
    # Reformat the data
    df['created_time'] = pd.to_datetime(df['created_time'], format = '%Y-%m-%d %H:%M:%S')
    # Split the word in each review
    df['title'].apply(lambda x: [w.strip() for w in x.split()])

    texts = df['title']
    texts = vectorizer.transform(texts)

    # initialize the output
    df['preds'] = 0

    classifiers = [
    ('LinearSVC', svm.LinearSVC()),
    ('LogisticReg', LogisticRegression()),
    ('SGD', SGDClassifier()),
    ('MultinomialNB', naive_bayes.MultinomialNB()),
    ('RandomForest', RandomForestClassifier())]
    
    for name, clf in classifiers:
        clf.fit(X, y)
        pred = clf.predict(texts)
        df[name] = pred
        df['preds'] = df['preds'] + df[name]
    df['preds'] = df['preds'].apply(lambda x: 0 if x < 3 else 1)
    
    # generate the predcition
    df.to_csv('generated_data/stock_comments_predicted.csv', index = False)
    print("Prediction has saved to file")
    
    return df

In [8]:
prediction = predict_with_bagging()

Prediction has saved to file


# Compute BI-index

In [9]:
def get_bi(row):
    
    pos = row[row == 1].count()
    neg = row[row == 0].count()
    bi = np.log((1 + pos) / (1 + neg))

    return bi

In [10]:
# Group reviews by date
grouped = prediction['preds'].groupby(prediction.created_time.dt.date)
# Generate bi score of each day
bi_index = grouped.apply(get_bi)
bi_index = bi_index.rename("BI")
# Get trading date according to Shanghai Index file
quotes = pd.read_csv('./data/sh000001.csv', parse_dates = ['date'])
quotes.set_index('date', inplace = True)
# Merge sentiment 
bi_index.index = pd.to_datetime(bi_index.index)
merged = pd.merge(bi_index, quotes, how = 'left', left_index = True, right_index = True)
merged.fillna(method = 'ffill', inplace = True)
# Save the bi index
merged.to_csv('generated_data/bi_idx.csv')